In [1]:
import pandas as pd
import sys
sys.path.append("..")
import utility.utility_functions as ut
import utility.data_prep_functions as dpf
import numpy as np  

In [2]:
week_end = ['Saturday','Sunday']
mid_week = ['Monday','Tuesday','Wednesday','Thursday','Friday']
seasons = {2.0:'Summer',0.0:'Winter',1.0:'Spring',3.0:'Fall'}
midnight = dpf.get_hours_label(0,7)
morning = dpf.get_hours_label(7,12)
noon_afternoon = dpf.get_hours_label(12,20)
night = dpf.get_hours_label(20,24)

In [3]:
df = ut.load_post_data_prep_all_zone()
df.dropna(inplace=True)

In [4]:
df.head()

,pickup_date,weekday,time_binned,Tmax,Tmin,Tavg,Tdep,HDD,CDD,Precipitation,new_snow,snow_depth,trip_distance,trip_time,speed,num_of_taxis,Zone
0,2019-01-01,Tuesday,00:00 - 00:59,58.0,39.0,48.5,13.3,16.0,0.0,0.06,0.0,0.0,2.543171,11.890650,12.167499,41,4
1,2019-01-01,Tuesday,01:00 - 01:59,58.0,39.0,48.5,13.3,16.0,0.0,0.06,0.0,0.0,2.842436,14.737821,11.223936,78,4
2,2019-01-01,Tuesday,02:00 - 02:59,58.0,39.0,48.5,13.3,16.0,0.0,0.06,0.0,0.0,2.235484,11.183333,11.346154,62,4
3,2019-01-01,Tuesday,03:00 - 03:59,58.0,39.0,48.5,13.3,16.0,0.0,0.06,0.0,0.0,2.640308,11.701795,12.651760,65,4
4,2019-01-01,Tuesday,04:00 - 04:59,58.0,39.0,48.5,13.3,16.0,0.0,0.06,0.0,0.0,2.635000,11.158889,13.186688,30,4


In [5]:
df['pickup_date'] = pd.to_datetime(df['pickup_date'])
df['month'] = df['pickup_date'].dt.month
df['Season'] = np.floor((df.month - 1 ) / 3)
df.Season.replace(seasons,inplace=True)

In [6]:

df = df.drop(df[df.trip_time < 5].index)
df = df.drop(['pickup_date','month'],axis=1)
df.reset_index(inplace=True,drop=True)

In [7]:
df.head()

,weekday,time_binned,Tmax,Tmin,Tavg,Tdep,HDD,CDD,Precipitation,new_snow,snow_depth,trip_distance,trip_time,speed,num_of_taxis,Zone,Season
0,Tuesday,00:00 - 00:59,58.0,39.0,48.5,13.3,16.0,0.0,0.06,0.0,0.0,2.543171,11.890650,12.167499,41,4,Winter
1,Tuesday,01:00 - 01:59,58.0,39.0,48.5,13.3,16.0,0.0,0.06,0.0,0.0,2.842436,14.737821,11.223936,78,4,Winter
2,Tuesday,02:00 - 02:59,58.0,39.0,48.5,13.3,16.0,0.0,0.06,0.0,0.0,2.235484,11.183333,11.346154,62,4,Winter
3,Tuesday,03:00 - 03:59,58.0,39.0,48.5,13.3,16.0,0.0,0.06,0.0,0.0,2.640308,11.701795,12.651760,65,4,Winter
4,Tuesday,04:00 - 04:59,58.0,39.0,48.5,13.3,16.0,0.0,0.06,0.0,0.0,2.635000,11.158889,13.186688,30,4,Winter


In [8]:
df.drop(['trip_distance','trip_time','speed'],axis=1,inplace=True)

In [9]:
import seaborn as sns
import plotly.express as px
import numpy as np

corr = df.corr().abs()

fig = px.imshow(corr, text_auto=True)
fig.update_layout(
    autosize=False,
    width=800,
    height=500,)

fig.show()

In [10]:
upper_tri = corr.where(np.triu(np.ones(corr.shape),k=1).astype(bool))
to_drop = [column for column in upper_tri.columns if any(upper_tri[column] > 0.85)]
to_drop
df1 = df.drop(to_drop, axis=1)


In [11]:
to_drop

['Tmin', 'Tavg', 'HDD']

In [12]:
main_df = ut.swap_first_and_last_col(df1)
main_df = ut.swap_first_and_last_col(main_df)

In [13]:
summer_mid_week_morning_df = main_df.loc[(main_df['Season'] == 'Summer')&(main_df['weekday'].isin(mid_week)) & (main_df['time_binned'].isin(morning))]
summer_mid_week_night_df = main_df.loc[(main_df['Season'] == 'Summer')&(main_df['weekday'].isin(mid_week)) & (main_df['time_binned'].isin(night))]
summer_mid_week_noon_afternoon_df = main_df.loc[(main_df['Season'] == 'Summer')&(main_df['weekday'].isin(mid_week)) & (main_df['time_binned'].isin(noon_afternoon))]
summer_mid_week_midnight_df = main_df.loc[(main_df['Season'] == 'Summer')&(main_df['weekday'].isin(mid_week)) & (main_df['time_binned'].isin(midnight))]

fall_mid_week_night_df = main_df.loc[(main_df['Season'] == 'Fall') & (main_df['weekday'].isin(mid_week)) & (main_df['time_binned'].isin(night))]
fall_mid_week_morning_df = main_df.loc[(main_df['Season'] == 'Fall') & (main_df['weekday'].isin(mid_week)) & (main_df['time_binned'].isin(morning))]
fall_mid_week_noon_afternoon_df = main_df.loc[(main_df['Season'] == 'Fall') & (main_df['weekday'].isin(mid_week)) & (main_df['time_binned'].isin(noon_afternoon))]
fall_mid_week_midnight_df = main_df.loc[(main_df['Season'] == 'Fall') & (main_df['weekday'].isin(mid_week)) & (main_df['time_binned'].isin(midnight))]

winter_mid_week_morning_df = main_df.loc[(main_df['Season'] == 'Winter') & (main_df['weekday'].isin(mid_week)) & (main_df['time_binned'].isin(morning))]
winter_mid_week_night_df = main_df.loc[(main_df['Season'] == 'Winter') & (main_df['weekday'].isin(mid_week)) & (main_df['time_binned'].isin(night))]
winter_mid_week_noon_afternoon_df = main_df.loc[(main_df['Season'] == 'Winter') & (main_df['weekday'].isin(mid_week)) & (main_df['time_binned'].isin(noon_afternoon))]
winter_mid_week_midnight_df = main_df.loc[(main_df['Season'] == 'Winter') & (main_df['weekday'].isin(mid_week)) & (main_df['time_binned'].isin(midnight))]

spring_mid_week_morning_df = main_df.loc[(main_df['Season'] == 'Spring') & (main_df['weekday'].isin(mid_week)) & (main_df['time_binned'].isin(morning))]
spring_mid_week_night_df = main_df.loc[(main_df['Season'] == 'Spring') & (main_df['weekday'].isin(mid_week)) & (main_df['time_binned'].isin(night))]
spring_mid_week_noon_afternoon_df = main_df.loc[(main_df['Season'] == 'Spring') & (main_df['weekday'].isin(mid_week)) & (main_df['time_binned'].isin(noon_afternoon))]
spring_mid_week_midnight_df = main_df.loc[(main_df['Season'] == 'Spring') & (main_df['weekday'].isin(mid_week)) & (main_df['time_binned'].isin(midnight))]

In [14]:
summer_week_end_morning_df = main_df.loc[(main_df['Season'] == 'Summer')&(main_df['weekday'].isin(week_end)) & (main_df['time_binned'].isin(morning))]
summer_week_end_night_df = main_df.loc[(main_df['Season'] == 'Summer')&(main_df['weekday'].isin(week_end)) & (main_df['time_binned'].isin(night))]
summer_week_end_noon_afternoon_df = main_df.loc[(main_df['Season'] == 'Summer')&(main_df['weekday'].isin(week_end)) & (main_df['time_binned'].isin(noon_afternoon))]
summer_week_end_midnight_df = main_df.loc[(main_df['Season'] == 'Summer')&(main_df['weekday'].isin(week_end)) & (main_df['time_binned'].isin(midnight))]

fall_week_end_night_df = main_df.loc[(main_df['Season'] == 'Fall') & (main_df['weekday'].isin(week_end)) & (main_df['time_binned'].isin(night))]
fall_week_end_morning_df = main_df.loc[(main_df['Season'] == 'Fall') & (main_df['weekday'].isin(week_end)) & (main_df['time_binned'].isin(morning))]
fall_week_end_noon_afternoon_df = main_df.loc[(main_df['Season'] == 'Fall') & (main_df['weekday'].isin(week_end)) & (main_df['time_binned'].isin(noon_afternoon))]
fall_week_end_midnight_df = main_df.loc[(main_df['Season'] == 'Fall') & (main_df['weekday'].isin(week_end)) & (main_df['time_binned'].isin(midnight))]

winter_week_end_morning_df = main_df.loc[(main_df['Season'] == 'Winter') & (main_df['weekday'].isin(week_end)) & (main_df['time_binned'].isin(morning))]
winter_week_end_night_df = main_df.loc[(main_df['Season'] == 'Winter') & (main_df['weekday'].isin(week_end)) & (main_df['time_binned'].isin(night))]
winter_week_end_noon_afternoon_df = main_df.loc[(main_df['Season'] == 'Winter') & (main_df['weekday'].isin(week_end)) & (main_df['time_binned'].isin(noon_afternoon))]
winter_week_end_midnight_df = main_df.loc[(main_df['Season'] == 'Winter') & (main_df['weekday'].isin(week_end)) & (main_df['time_binned'].isin(midnight))]

spring_week_end_morning_df = main_df.loc[(main_df['Season'] == 'Spring') & (main_df['weekday'].isin(mid_week)) & (main_df['time_binned'].isin(morning))]
spring_week_end_night_df = main_df.loc[(main_df['Season'] == 'Spring') & (main_df['weekday'].isin(week_end)) & (main_df['time_binned'].isin(night))]
spring_week_end_noon_afternoon_df = main_df.loc[(main_df['Season'] == 'Spring') & (main_df['weekday'].isin(week_end)) & (main_df['time_binned'].isin(noon_afternoon))]
spring_week_end_midnight_df = main_df.loc[(main_df['Season'] == 'Spring') & (main_df['weekday'].isin(week_end)) & (main_df['time_binned'].isin(midnight))]

In [15]:
df_dict = {
            'summer_mid_week_morning_df': summer_mid_week_morning_df,
            'summer_mid_week_night_df': summer_mid_week_night_df ,
            'summer_mid_week_noon_afternoon_df': summer_mid_week_noon_afternoon_df,
            'summer_mid_week_midnight_df': summer_mid_week_midnight_df,
            'fall_mid_week_night_df': fall_mid_week_night_df,
            'fall_mid_week_morning_df':fall_mid_week_morning_df ,
            'fall_mid_week_noon_afternoon_df':fall_mid_week_noon_afternoon_df ,
            'fall_mid_week_midnight_df': fall_mid_week_midnight_df,
            'winter_mid_week_morning_df': winter_mid_week_morning_df,
            'winter_mid_week_night_df': winter_mid_week_night_df,
            'winter_mid_week_noon_afternoon_df': winter_mid_week_noon_afternoon_df,
            'winter_mid_week_midnight_df':winter_mid_week_midnight_df ,
            'spring_mid_week_morning_df': spring_mid_week_morning_df,
            'spring_mid_week_night_df': spring_mid_week_night_df,
            'spring_mid_week_noon_afternoon_df': spring_mid_week_noon_afternoon_df,
            'spring_mid_week_midnight_df': spring_mid_week_midnight_df,
            'summer_week_end_morning_df' : summer_week_end_morning_df,
            'summer_week_end_night_df': summer_week_end_night_df,
            'summer_week_end_noon_afternoon_df': summer_week_end_noon_afternoon_df,
            'summer_week_end_midnight_df': summer_week_end_midnight_df,
            'fall_week_end_night_df': fall_week_end_night_df,
            'fall_week_end_morning_df': fall_week_end_morning_df,
            'fall_week_end_noon_afternoon_df': fall_week_end_noon_afternoon_df,
            'fall_week_end_midnight_df': fall_week_end_midnight_df,
            'winter_week_end_morning_df': winter_week_end_morning_df,
            'winter_week_end_night_df': winter_week_end_night_df,
            'winter_week_end_noon_afternoon_df' : winter_week_end_noon_afternoon_df,
            'winter_week_end_midnight_df': winter_week_end_midnight_df,
            'spring_week_end_morning_df': spring_week_end_morning_df,
            'spring_week_end_night_df': spring_week_end_night_df,
            'spring_week_end_noon_afternoon_df': spring_week_end_noon_afternoon_df,
            'spring_week_end_midnight_df': spring_week_end_midnight_df,
}

In [16]:
import rmse_script 

c:\Users\barka\AppData\Local\Programs\Python\Python310\lib\site-packages\xgboost\compat.py:36: FutureWarning:

pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.



In [18]:
for key, value in df_dict.items():
    x,y,t = rmse_script.get_rmse_for_df(value,key)
    print(f'Group {key} RMSE of {x:.2f} and test RMSE of {t:.2f} target feature std of {y:.2f}')

Group summer_mid_week_morning_df RMSE of 99.97 and test RMSE of 98.97 target feature std of 115.34
Group summer_mid_week_night_df RMSE of 124.56 and test RMSE of 131.14 target feature std of 132.93
Group summer_mid_week_noon_afternoon_df RMSE of 121.13 and test RMSE of 123.10 target feature std of 140.72
Group summer_mid_week_midnight_df RMSE of 41.56 and test RMSE of 40.39 target feature std of 44.52
Group fall_mid_week_night_df RMSE of 146.32 and test RMSE of 148.04 target feature std of 156.26
Group fall_mid_week_morning_df RMSE of 108.52 and test RMSE of 109.65 target feature std of 132.58
Group fall_mid_week_noon_afternoon_df RMSE of 134.76 and test RMSE of 135.60 target feature std of 158.33
Group fall_mid_week_midnight_df RMSE of 45.22 and test RMSE of 44.42 target feature std of 48.35
Group winter_mid_week_morning_df RMSE of 121.89 and test RMSE of 121.29 target feature std of 142.96
Group winter_mid_week_night_df RMSE of 155.21 and test RMSE of 155.49 target feature std of 166